# Análise dos dados abertos da ANTT

Os dados utilizados nesse notebook vêm da página de Dados Abertos da ANTT (Agência Nacional de Transportes Terrestres), mais especificamente do grupo "Rodovias", disponível no [link](https://dados.antt.gov.br/group/rodovias]). Para saber mais sobre o Portal de Dados Abertos da ANTT veja o seguinte [link](https://dados.antt.gov.br/about).

## Objetivo

O principal objetivo desse notebook é criar vizualizações para verificar a consistência dos dados fornecidos à ANTT pelas concessionárias de cada trecho de rodovia. Por exemplo, verficar os traçados das rodovias.

# Análises e Vizualizações


## Observações sobre o notebook

Este notebook está no [GitHub](https://github.com/GuiBatalhoti/Dados_ANTT) e está sendo desenvolvido de forma local, porém devido à necessidade de acesso de outros colaboradores, o notebook também está no Google Colab.

Na necessidade de execução pelo Colab, executar a seguinte célula:

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import folium
import shapely
# from shapely import wkt
# from shapely import frechet_distance

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Abrindo os dados de KM de Pista

Especificações em: data/km_pista/quilometro_principal_dicionario_dados.pdf

In [3]:
pista_df = pd.read_csv('data/km_pista/dados_dos_quilometro_principal.csv', sep=';', encoding='latin1')
print(pista_df.info())
pista_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255127 entries, 0 to 255126
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   concessionaria  255127 non-null  object
 1   ano_do_pnv_snv  255127 non-null  int64 
 2   rodovia         255127 non-null  object
 3   km_m            255127 non-null  object
 4   sentido         255127 non-null  object
 5   latitude        255127 non-null  object
 6   longitude       255127 non-null  object
dtypes: int64(1), object(6)
memory usage: 13.6+ MB
None


,concessionaria,ano_do_pnv_snv,rodovia,km_m,sentido,latitude,longitude
0,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,"22,600",Decrescente,"-22,999214","-46,518242"
1,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,"22,400",Decrescente,"-22,997770","-46,517069"
2,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,"22,200",Decrescente,"-22,996627","-46,515558"
3,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,"22,000",Decrescente,"-22,995466","-46,514080"
4,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,"21,800",Decrescente,"-22,994674","-46,512898"


In [4]:
pista_df['km_m'] = pista_df['km_m'].str.replace(',', '.').astype(float)
pista_df['ano_do_pnv_snv'] = pista_df['ano_do_pnv_snv'].astype(int)
pista_df['latitude'] = pista_df['latitude'].str.replace(',', '.').astype(float)
pista_df['longitude'] = pista_df['longitude'].str.replace(',', '.').astype(float)

print(pista_df.info())
pista_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255127 entries, 0 to 255126
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   concessionaria  255127 non-null  object 
 1   ano_do_pnv_snv  255127 non-null  int64  
 2   rodovia         255127 non-null  object 
 3   km_m            255127 non-null  float64
 4   sentido         255127 non-null  object 
 5   latitude        255127 non-null  float64
 6   longitude       255127 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 13.6+ MB
None


,concessionaria,ano_do_pnv_snv,rodovia,km_m,sentido,latitude,longitude
0,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,22.6,Decrescente,-22.999214,-46.518242
1,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,22.4,Decrescente,-22.997770,-46.517069
2,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,22.2,Decrescente,-22.996627,-46.515558
3,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,22.0,Decrescente,-22.995466,-46.514080
4,AUTOPISTA FERNÃO DIAS,2007,BR-381/SP,21.8,Decrescente,-22.994674,-46.512898


Colocando os dados de forma "Crua" no mapa, desconsiderando o sentido

In [5]:
mapa = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

cores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'gray', 'black']
rodovia = pista_df['rodovia'].iloc[0]
sentido = pista_df['sentido'].iloc[0]
pontos_linha  = []
for i, linha in pista_df.iterrows():
    if linha['rodovia'] != rodovia:
        mapa.add_child(folium.PolyLine(locations=pontos_linha, 
                                       color=cores[np.random.randint(0, len(cores))], 
                                       alpha=0.7,
                                       weight=3, 
                                       popup=folium.Popup(f"{rodovia}\n{linha['concessionaria']}")))
        rodovia = linha['rodovia']
        sentido = linha['sentido']
        pontos_linha = []
    pontos_linha.append([linha['latitude'], linha['longitude']])

if mapa.save('output/mapa_rodovias_cru.html'):
    print('Mapa salvo com sucesso, em output/mapa_rodovias_cru.html')

É possível obervar que algumas rodovias apresentam "riscos" que  cortam todo o seu trajeto, o que pode ser um erro de preenchimento dos dados. Exemplos:
- BR-163/MS (RIOSP)
- BR-163/MT (CRO)
- BR-163/MT (ECO050)
- BR-364/MT (CRO)
- BR-70/MT (CRO)
- BR-116/RJ (RODOVIA DO AÇO)

Em alguns casos, podemos observar que o "risco" atravessa todo o comprimenro da rodovia, que pode estar relacionado à desconsideração do sentido da rodovia. Exemplos:
- BR-116/SP (AUTOPISTA REGIS BITTENCOURT)
- BR-262/MG (CRO)
- BR-153/MG (CONCEBRA)

Transformando os dados em GeoDataFrame para melhor manipulação

In [6]:
def df2gdf(df: pd.DataFrame, ignora_sentido: bool) -> gpd.GeoDataFrame:
    df.sort_values(['rodovia', 'km_m', 'sentido'], inplace=True, ignore_index=True)
    df_return = {
        "rodovia": [],
        "sentido": [],
        "concessionaria": [],
        "km": [],
        "ano_do_pnv_snv": [],
        "geometry": []
    }
    pontos = []

    linha_anterior = df.iloc[0]
    for i, linha in df.iterrows():
        if linha['rodovia'] != linha_anterior['rodovia'] or (linha['sentido'] != linha_anterior['sentido'] and not ignora_sentido):
            df_return["rodovia"].append(linha_anterior['rodovia'])
            df_return["sentido"].append(linha_anterior['sentido'])
            df_return["concessionaria"].append(linha_anterior['concessionaria'])
            df_return["km"].append(df[(df['rodovia'] == linha_anterior['rodovia']) &
                                  (df['sentido'] == linha_anterior['sentido']) &
                                  (df['concessionaria'] == linha_anterior['concessionaria'])]['km_m'].max())
            df_return["ano_do_pnv_snv"].append(linha_anterior['ano_do_pnv_snv'])
            if len(pontos) > 1:
                df_return["geometry"].append(shapely.geometry.LineString(pontos))
            else:
                df_return["geometry"].append(shapely.geometry.Point(pontos[0]))
            pontos = []
            
        pontos.append((linha['longitude'], linha['latitude']))
        linha_anterior = linha


    return gpd.GeoDataFrame(df_return, crs="EPSG:4326")

In [7]:
pista_gdf = df2gdf(pista_df, ignora_sentido=True)
# pista_gdf_sentido = df2gdf(pista_df, ignora_sentido=False)

In [8]:
pista_gdf

,rodovia,sentido,concessionaria,km,ano_do_pnv_snv,geometry
0,092/PR,Decrescente,LITORAL PIONEIRO,324.090,2019,"LINESTRING (-49.73240 -24.25181, -49.73472 -24..."
1,151/PR,Crescente,LITORAL PIONEIRO,318.022,2019,"LINESTRING (-49.47956 -24.11651, -49.48566 -24..."
2,239/PR,Decrescente,LITORAL PIONEIRO,12.359,2019,"LINESTRING (-49.37777 -24.11891, -49.38658 -24..."
3,407/PR,Crescente,LITORAL PIONEIRO,17.990,2019,"LINESTRING (-48.57749 -25.56884, -48.57285 -25..."
4,408/PR,Crescente,LITORAL PIONEIRO,21.999,2019,"LINESTRING (-48.75282 -25.52557, -48.76237 -25..."
...,...,...,...,...,...,...
94,CONTORNO SUL - FS1/BA,Decrescente,VIA BAHIA,7.900,2003,"LINESTRING (-38.98886 -12.26814, -38.98886 -12..."
95,CW-116/PR,Decrescente,AUTOPISTA REGIS BITTENCOURT,18.500,2017,"LINESTRING (-49.04491 -25.34556, -49.04491 -25..."
96,Contorno de Iconha/ES,Decrescente,ECO101,7.044,2020,"LINESTRING (-40.78341 -20.78697, -40.78341 -20..."
97,Contorno de Uberlândia/MG,Decrescente,ECO050,21.000,2017,"LINESTRING (-48.25685 -18.85986, -48.25692 -18..."


## Colocando todos os dados em um mapa interativo

In [15]:
mapa = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

cores = list(mcolors.TABLEAU_COLORS.keys())

folium.GeoJson(pista_gdf, 
               name='Pista Principal',
               style_function= lambda x: {
                     'color': 'darkblue',
                     'weight': 3,
                     'opacity': 0.6
               }).add_to(mapa)
folium.LayerControl().add_to(mapa)


mapa.save('output/mapa_rodovias.html')

In [10]:
# brasil = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BRA_shp.zip", layer="gadm41_BRA_1")
# cores = list(mcolors.TABLEAU_COLORS.keys())

# fig = plt.figure(figsize=(20,20), dpi=100)
# ax = fig.add_subplot(111)
# ax.set_title("Malha Rodoviária do Brasil, análise qualitativa")

# brasil.plot(ax=ax, color='lightgray', edgecolor='black')
# for i, (index, linha) in enumerate(pista_gdf.iterrows()):
#     if linha['rodovia'].startswith('BR'):
#         pista_gdf.iloc[[i]].plot(ax=ax, color=cores[i % len(cores)], linewidth=2, alpha=0.7, label=f"{linha['rodovia']} - {linha['concessionaria']}")

# plt.legend(title="Rodovias - Concessionárias", loc="upper left")
# plt.show()

In [11]:
# brasil = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BRA_shp.zip", layer="gadm41_BRA_1")
# cores = list(mcolors.TABLEAU_COLORS.keys())

# fig = plt.figure(figsize=(20,20), dpi=100)
# ax = fig.add_subplot(111)
# ax.set_title("Malha Rodoviária do Brasil, análise qualitativa")

# brasil.plot(ax=ax, color='lightgray', edgecolor='black')
# for i, (index, linha) in enumerate(pista_gdf.iterrows()):
#     pista_gdf.iloc[[i]].plot(ax=ax, color=cores[i % len(cores)], linewidth=2, alpha=0.7, label=f"{linha['rodovia']} - {linha['concessionaria']}")

# plt.legend(title="Rodovias - Concessionárias", loc="upper left")
# plt.show()

In [ ]:
brasil = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BRA_shp.zip", layer="gadm41_BRA_1")
cores = list(mcolors.TABLEAU_COLORS.keys())

fig = plt.figure(figsize=(10,10))

gs = fig.add_gridspec(1, 2, wspace=0, width_ratios=[1, 1], hspace=0, height_ratios=[1])
(ax1, ax2) = gs.subplots(sharex=True, sharey=True)

ax1.set_title("Rodovias Principais")
ax2.set_title("Entruncamentos e Contornos")

brasil.plot(ax=ax1, color='lightgray', edgecolor='black')
brasil.plot(ax=ax2, color='lightgray', edgecolor='black')
for i, (index, linha) in enumerate(pista_gdf.iterrows()):
    cor = cores[i % len(cores)]
    if linha['rodovia'].startswith('BR'):
        pista_gdf.iloc[[i]].plot(ax=ax1, color=cor, linewidth=2, alpha=0.7)
    pista_gdf.iloc[[i]].plot(ax=ax2, color=cor, linewidth=2, alpha=0.7)

# plt.legend(title="Rodovias - Concessionárias", loc="upper left")
plt.savefig('output/rodovias.pdf')

In [ ]:
pista_gdf['estado'] = pista_gdf['rodovia'].map(lambda x: x.split('/')[1])
pista_gdf = pista_gdf[['rodovia', 'estado', 'sentido', 'concessionaria', 'km', 'ano_do_pnv_snv', 'geometry']]
pista_gdf

comparação dos dados do OpenStreetMap com os dados da ANTT da BR-163

In [ ]:
df = pd.read_csv('data/rodovias_MT-MS/points.csv')
df['geometry'] = df.apply(wkt.loads)
pista_osm_gdf = gpd.GeoDataFrame(df, crs="")
pista_osm_gdf.set_crs(epsg=3857, inplace=True)
pista_osm_gdf.to_crs(epsg=4326, inplace=True)
#adicionar os pontos à uma lineString
pista_osm_line = geom.LineString(pista_osm_gdf['geometry'].values)
pista_osm_gdf = gpd.GeoDataFrame({'geometry': [pista_osm_line]}, crs="EPSG:4326")
pista_osm_gdf

Calculando a distância entre os pontos do OpenStreetMap e da ANTT

In [ ]:
rodovias_mt_ms = pista_gdf[pista_gdf['estado'].isin(['MT', 'MS'])]
rodovias_mt_ms

In [16]:
line_rodo_mt_ms = rodovias_mt_ms.unary_union
distancia = frechet_distance(line_rodo_mt_ms, pista_osm_line)

In [ ]:
print(f"A distância de Fréchet entre as rodovias de MT/MS e a linha do OSM é de {distancia}.")

In [ ]:
brasil = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_BRA_shp.zip", layer="gadm41_BRA_1")

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.set_title("Rodovias Principais")

brasil.plot(ax=ax, color='lightgray', edgecolor='black')
pista_osm_gdf.plot(ax=ax, color='red', alpha=0.7, label="BR-163 - OSM")
pista_gdf[pista_gdf['estado'] == 'MT'].plot(ax=ax, color='blue', alpha=0.7, label="BR-163 - ANTT")
